In [20]:
import spacy 
import re 
import pandas as pd
from spacy.lang.en import English
from collections import Counter
from itertools import chain
import ahocorasick
import ast
from analyze_m_data import get_occupations_list_en, get_occupations_list
from spacy.lang.de import German
from spacy.lang.it import Italian
from spacy.lang.es import Spanish 
import matplotlib.pyplot as plt
import itertools
from collections import Counter
from operator import itemgetter

In [25]:
def stem_titles(titles):
  from nltk.stem import SnowballStemmer 
  stemmer = SnowballStemmer("italian") # Choose a language
  stems = []
  for title in titles:
     stem = stemmer.stem(title)
     stems.append(stem)
  return stems
it_occ_titles = pd.read_csv('/mount/arbeitsdaten/studenten2/caulfiea/masters_thesis/it_occ_list_from_en.csv', encoding='utf-8')
it_occ_list = it_occ_titles['occupations'].to_list()
stems = stem_titles(it_occ_list)
print(stems[:10])

['astrazion', 'contabil', 'acrob', 'attor', 'attric', 'attuar', 'agopressor', 'agopuntur', 'indirizz', 'giudice arbitr']


In [26]:
def get_occupations(df):
    new_df = df
    # print(unique_gender_counts)
    # number of biographies that returned no intro 
    print("num intros unavailable")
    print((new_df['intro'] == 'No intro available').sum())
    # remove biographies with no intro 
    new_df = new_df[new_df.intro != 'No intro available']
    print(new_df.shape[0])
    small_df = new_df.copy()
    # lowercase intro strings
    small_df['intro'] = small_df['intro'].str.casefold()
    small_df['title'] = small_df['title'].str.casefold()
    # small_df['first_sentence'] = small_df['first_sentence'].str.casefold()
    # the lists in the occupations column were not actual lists but rather string literals
    # this code converts them back into lists  
    # small_df['occupations'] = small_df['occupations'].apply(ast.literal_eval)

    # get a set of occupation words 
    full_unique_occupations = set(stem_titles(it_occ_list))
    # full_unique_occupations = set(stem_titles(es_occ_list))
    # full_unique_occupations = get_occupations_list('/mount/arbeitsdaten/studenten2/caulfiea/masters_thesis/de_occ_titles.csv', gender)
    # full_unique_occupations = get_occupations_list_en('/mount/arbeitsdaten/studenten2/caulfiea/masters_thesis/it_occ_list_full.csv')
    # full_unique_occupations = get_occupations_list_en('/mount/arbeitsdaten/studenten2/caulfiea/masters_thesis/es_occ_list_unique.csv')
    small_df['intro'] = small_df['intro'].astype(str)
    small_df['title'] = small_df['title'].astype(str)
    
    print(small_df.dtypes)
    # want to find overlapping terms in the intros column and the wikidata occupation words set 
    automaton = ahocorasick.Automaton()
    for id, term in enumerate(full_unique_occupations):
        automaton.add_word(term, (term))
    automaton.remove_word('-')
    automaton.make_automaton()
    def find_terms_en(text):
        return list(term for _, term in automaton.iter(text))
    def find_terms_de(text):
        words = list(re.findall(r'\b\w+\b', text.lower()))  # Tokenize text into words
        matches = list(term for _, term in automaton.iter(text) if term.lower() in words)
        return matches
    def find_terms_stems1(text):
      matching_tokens = []
      substrings = list(full_unique_occupations)
      tokens = text.split()
      for token in tokens:
        if any(sub in token for sub in substrings):
          matching_tokens.append(token)
      return matching_tokens
    
    def find_terms_stems(text):
      # ner = spacy.load('en_core_web_sm') 
      # ner = spacy.load('de_core_news_sm')
      ner = spacy.load('it_core_news_sm') 
      # ner = spacy.load('es_core_news_sm')
      doc = ner(text) 
      occupational_titles = list(full_unique_occupations)
      matched_occupations = []
      for ent in doc: 
        if any(sub in ent.text.casefold() for sub in occupational_titles):
          if not ent.ent_type_:
              # matched_occupations.append(ent.text)
              matched_occupations.append({ent.text: ent.morph.get("Gender")})
        # print(ent.text, ent.pos_, ent.dep_)
          # print(ent,ent.morph.get("Gender"))
      return (matched_occupations)
    
    # small_df['overlapping_occupations'] = small_df['intro'].apply(find_terms_stems)
    # small_df['overlapping_occupations_sentence1'] = small_df['first_sentence'].apply(find_terms_stems)
    # want to see if male occupation names show up in title field as last names 
    # small_df['occupation_in_last_name'] = small_df['title'].apply(find_terms_stems)
    small_df['overlapping_occupations_sentence1'] = small_df['overlapping_occupations_sentence1'].apply(ast.literal_eval)
    occ_counts_counter = Counter(
    key
    for row in small_df['overlapping_occupations_sentence1']
    for d in row
    for key in d.keys()
    )

    occ_gender_counts_counter = Counter(
    value
    for row in small_df['overlapping_occupations_sentence1']
    for d in row
    for values in d.values()
    for value in values
    )
   
    # overlapping_occ_counts = Counter(chain.from_iterable([small_df['overlapping_occupations_sentence1'].keys()]))
    # print(small_df[['wikidata_code','intro', 'overlapping_occupations']].head(20))
    print(occ_counts_counter)
    print(occ_gender_counts_counter)
    smaller_df = small_df[small_df['overlapping_occupations_sentence1'].map(len)>0]
    # smaller_df = smaller_df[smaller_df['overlapping_occupations'].map(len)>0]

    print("length of dataset:")
    print(small_df.shape[0])
    print("first sentences with occ title found:", smaller_df.shape[0])
    

In [18]:
it_female_df = pd.read_csv('/mount/arbeitsdaten/studenten2/caulfiea/masters_thesis/sets_for_analysis/it/it_female_occ_all.csv', encoding='utf-8')
get_occupations(it_female_df)

num intros unavailable
0
7852
Unnamed: 0                            int64
wikidata_code                        object
title                                object
intro                                object
gender                               object
occupations                          object
first_sentence                       object
overlapping_occupations              object
overlapping_occupations_sentence1    object
occupation_in_last_name              object
dtype: object
Counter({'attrice': 1417, 'cestista': 921, 'maggio': 743, 'politica': 681, 'scrittrice': 621, 'calciatrice': 536, 'cantante': 451, 'giornalista': 444, 'conduttrice': 423, 'per': 283, 'modella': 250, 'doppiatrice': 248, 'poetessa': 177, 'direttrice': 111, 'allenatrice': 110, 'nobildonna': 107, 'storica': 105, 'ballerina': 99, 'saggista': 90, 'personaggio': 74, 'doppiaggio': 72, 'canadese': 70, 'deputata': 68, 'critica': 68, 'artista': 60, 'argentina': 57, 'insegnante': 54, 'portiere': 45, 'figlia': 42, 'musicist

In [27]:
it_female_df2 = pd.read_csv('/mount/arbeitsdaten/studenten2/caulfiea/masters_thesis/sets_for_analysis/it/it_female_occ_2.csv', encoding='utf-8')
get_occupations(it_female_df2)

num intros unavailable
0
7852
Unnamed: 0                            int64
wikidata_code                        object
title                                object
intro                                object
gender                               object
occupations                          object
first_sentence                       object
overlapping_occupations              object
overlapping_occupations_sentence1    object
occupation_in_last_name              object
dtype: object
Counter({'attrice': 1417, 'cestista': 921, 'politica': 681, 'scrittrice': 621, 'calciatrice': 536, 'cantante': 451, 'giornalista': 444, 'conduttrice': 423, 'per': 283, 'modella': 250, 'doppiatrice': 248, 'poetessa': 177, 'traduttrice': 133, 'schermitrice': 125, 'direttrice': 111, 'allenatrice': 110, 'nobildonna': 107, 'autrice': 107, 'storica': 105, 'ballerina': 99, 'vincitrice': 97, 'saggista': 90, 'cantautrice': 89, 'pittrice': 87, 'sceneggiatrice': 77, 'giocatrice': 75, 'personaggio': 74, 'doppiaggio': 72, '

In [19]:
it_male_df = pd.read_csv('/mount/arbeitsdaten/studenten2/caulfiea/masters_thesis/sets_for_analysis/it/it_male_occ_all.csv', encoding='utf-8')
get_occupations(it_male_df)

num intros unavailable
0
7784
Unnamed: 0                            int64
wikidata_code                        object
title                                object
intro                                object
gender                               object
occupations                          object
first_sentence                       object
overlapping_occupations              object
overlapping_occupations_sentence1    object
occupation_in_last_name              object
dtype: object
Counter({'politico': 1213, 'calciatore': 1179, 'maggio': 798, 'scrittore': 423, 'allenatore': 398, 'giornalista': 379, 'attore': 332, 'pittore': 266, 'cestista': 231, 'per': 195, 'poeta': 189, 'storico': 186, 'avvocato': 174, 'compositore': 163, 'architetto': 146, 'sportivo': 139, 'critico': 123, 'direttore': 120, 'scultore': 118, 'portiere': 113, 'medico': 112, 'saggista': 95, 'produttore': 87, 'conduttore': 85, 'cantante': 77, 'seconda': 76, 'musicista': 74, 'autore': 71, 'cantautore': 71, 'presidente': 68, '

In [28]:
it_male_df2 = pd.read_csv('/mount/arbeitsdaten/studenten2/caulfiea/masters_thesis/sets_for_analysis/it/it_male_occ_2.csv', encoding='utf-8')
get_occupations(it_male_df2)

num intros unavailable
0
7784
Unnamed: 0                            int64
wikidata_code                        object
title                                object
intro                                object
gender                               object
occupations                          object
first_sentence                       object
overlapping_occupations              object
overlapping_occupations_sentence1    object
occupation_in_last_name              object
dtype: object
Counter({'politico': 1213, 'calciatore': 1179, 'scrittore': 423, 'allenatore': 398, 'giornalista': 379, 'attore': 332, 'pittore': 266, 'cestista': 231, 'per': 195, 'poeta': 189, 'storico': 186, 'imprenditore': 177, 'avvocato': 174, 'compositore': 163, 'giocatore': 154, 'architetto': 146, 'sportivo': 139, 'critico': 123, 'direttore': 120, 'scultore': 118, 'portiere': 113, 'medico': 112, 'saggista': 95, 'produttore': 87, 'conduttore': 85, 'cantante': 77, 'seconda': 76, 'musicista': 74, 'autore': 71, 'aviatore': 7